# 🌟 00: LangGraph 기본 개념 맛보기

## 💡 5분만에 이해하는 State와 Node!

LangGraph의 핵심은 **State**(상태)와 **Node**(노드) 두 개념이에요.  
복잡한 걸 배우기 전에 이 둘이 뭔지 간단하게 체험해보자!

## 🎯 이 노트북에서 배우는 것들

### ✅ 핵심 학습 목표
1. **State (상태)**: 워크플로우의 데이터 저장소 개념과 TypedDict 활용
2. **Node (노드)**: State를 처리하는 함수의 구조와 역할
3. **StateGraph**: 노드들을 연결해서 워크플로우 만드는 방법
4. **워크플로우 실행**: `compile()` → `invoke()` 실행 패턴
5. **그래프 시각화**: `visualize_graph()`로 구조 파악하기
6. **조건부 분기**: 입력에 따라 다른 경로로 처리하는 방법

### 💡 실습 패턴
각 섹션은 **정의 → 테스트 → 시각화 → 분석** 순서로 진행됩니다.

### ⏱️ 예상 소요시간: 5분

In [ ]:
# 필요한 라이브러리 설치
!pip install -q langgraph langchain langchain-teddynote
!pip install -q grandalf matplotlib networkx pyppeteer

# 기본 import
from typing import TypedDict
from langgraph.graph import StateGraph, END, START
from langchain_teddynote.graphs import visualize_graph

print("✅ LangGraph 준비 완료!")

## 🎯 State란 뭘까?

**State = 워크플로우에서 공유하는 데이터 주머니**

- 처음에 뭔가 넣고 → 노드들이 처리하면서 업데이트 → 최종 결과 나옴
- 딕셔너리처럼 생겼는데 타입이 정해져 있어!

In [ ]:
# 가장 간단한 State 정의
class SimpleState(TypedDict):
    input: str      # 사용자가 넣을 데이터
    output: str     # 최종 결과가 나올 곳

# 예시 State
example_state = {
    "input": "안녕하세요",
    "output": ""  # 아직 빈 상태
}

print("🗂️ SimpleState 정의 완료!")
print(f"📝 예시: {example_state}")

## 🔧 Node란 뭘까?

**Node = State를 받아서 처리하고 업데이트하는 함수**

- State를 파라미터로 받음 → 뭔가 처리함 → 업데이트된 State 리턴
- 함수 하나하나가 처리 단계가 됨!

In [ ]:
# 가장 간단한 Node 함수
def hello_node(state: SimpleState) -> SimpleState:
    """인사해주는 노드"""
    user_input = state["input"]
    greeting = f"🤖 안녕하세요, '{user_input}'라고 말씀하셨군요!"
    
    return {
        "input": user_input,      # 원래 input 유지
        "output": greeting        # output 업데이트!
    }

# 테스트해보기
test_state = {"input": "반가워요", "output": ""}
result_state = hello_node(test_state)

print("✅ hello_node 함수 정의 완료!")
print(f"🧪 테스트 입력: {test_state}")
print(f"🧪 테스트 출력: {result_state}")

## 🚀 첫 번째 워크플로우 만들기

이제 **StateGraph**로 노드를 연결해서 워크플로우를 만들어보자!

In [ ]:
# 1개 노드로 구성된 가장 간단한 워크플로우
workflow = StateGraph(SimpleState)

# 노드 추가
workflow.add_node("hello", hello_node)

# 시작점과 끝점 설정
workflow.set_entry_point("hello")
workflow.add_edge("hello", END)

# 실행 가능한 앱으로 컴파일
app = workflow.compile()

print("🎯 첫 번째 워크플로우 완성!")
print("   흐름: START → hello → END")

In [ ]:
# 첫 번째 그래프 시각화
print("📊 첫 번째 워크플로우 구조:")
visualize_graph(app)

In [ ]:
# 워크플로우 실행해보기
initial_state = {
    "input": "LangGraph 배우는 중!",
    "output": ""
}

print("🚀 워크플로우 실행:")
print(f"📥 초기 상태: {initial_state}")

final_state = app.invoke(initial_state)
print(f"📤 최종 상태: {final_state}")
print(f"\n🎉 결과: {final_state['output']}")

## 🔗 3개 노드 연결해보기

이제 노드를 3개 만들어서 순서대로 실행해보자!  
첫 번째 노드 → 두 번째 노드 → 세 번째 노드 → 끝

In [ ]:
# 추가 노드들 정의
def process_node(state: SimpleState) -> SimpleState:
    """입력을 처리하는 노드"""
    user_input = state["input"]
    processed = f"⚙️ '{user_input}'을(를) 처리했습니다"
    
    return {
        "input": user_input,
        "output": processed
    }

def format_node(state: SimpleState) -> SimpleState:
    """결과를 예쁘게 포맷하는 노드"""
    current_output = state["output"]
    formatted = f"✨ [최종결과] {current_output} ✨"
    
    return {
        "input": state["input"],
        "output": formatted
    }

print("✅ 추가 노드들 정의 완료!")

In [ ]:
# 3개 노드를 연결한 워크플로우
multi_workflow = StateGraph(SimpleState)

# 노드들 추가
multi_workflow.add_node("hello", hello_node)
multi_workflow.add_node("process", process_node)
multi_workflow.add_node("format", format_node)

# 연결: hello → process → format → END
multi_workflow.set_entry_point("hello")
multi_workflow.add_edge("hello", "process")
multi_workflow.add_edge("process", "format")
multi_workflow.add_edge("format", END)

# 컴파일
multi_app = multi_workflow.compile()

print("🔗 3단계 워크플로우 완성!")
print("   흐름: hello → process → format → END")

In [ ]:
# 3단계 워크플로우 시각화  
print("📊 3단계 워크플로우 구조:")
visualize_graph(multi_app)

In [ ]:
# 3단계 워크플로우 실행 테스트
test_input = {
    "input": "노드 체인 테스트",
    "output": ""
}

print("🧪 3단계 워크플로우 실행:")
print(f"📥 입력: '{test_input['input']}'")
print("-" * 50)

final_result = multi_app.invoke(test_input)

print(f"📤 최종 결과: {final_result['output']}")
print("\n🎯 3개 노드가 순서대로 처리했어요!")

In [ ]:
# 각 노드별 처리 과정 추적
print("🔬 각 노드가 State를 어떻게 변경하는지 단계별 확인:")
print("=" * 60)

initial_state = {"input": "단계별 추적", "output": ""}
print(f"🏁 초기 상태: {initial_state}")

after_hello = hello_node(initial_state)
print(f"1️⃣ hello_node 후: {after_hello}")

after_process = process_node(after_hello)
print(f"2️⃣ process_node 후: {after_process}")

after_format = format_node(after_process)
print(f"3️⃣ format_node 후: {after_format}")

print("\n💡 관찰: 각 노드가 State의 'output' 필드를 업데이트합니다!")

## 🤔 조건부 분기 맛보기

입력 길이에 따라 다른 경로로 보내보자!  
- 짧으면 → 간단 처리
- 길면 → 복잡 처리

In [ ]:
# 조건부 분기를 위한 상태와 함수들 정의
class BranchState(TypedDict):
    input: str
    output: str
    path: str  # 어떤 경로로 처리됐는지 추적

def decide_path(state: BranchState):
    """입력 길이에 따라 경로 결정"""
    input_text = state["input"]
    if len(input_text) <= 5:
        return "simple"
    else:
        return "complex"

def simple_process(state: BranchState):
    """짧은 입력용 간단 처리"""
    input_text = state["input"]
    result = f"🟢 간단 처리: '{input_text}' → 빠른 응답"
    
    return {
        "input": input_text,
        "output": result,
        "path": "simple"
    }

def complex_process(state: BranchState):
    """긴 입력용 복잡 처리"""
    input_text = state["input"]
    result = f"🔵 복잡 처리: '{input_text}' → 상세 분석 완료"
    
    return {
        "input": input_text,
        "output": result,
        "path": "complex"
    }

print("✅ 조건부 분기용 함수들 정의 완료!")

In [ ]:
# 조건부 분기 워크플로우
branch_workflow = StateGraph(BranchState)

# 노드들 추가
branch_workflow.add_node("simple", simple_process)
branch_workflow.add_node("complex", complex_process)

# 시작점은 조건부 엣지
branch_workflow.set_conditional_entry_point(
    decide_path,
    {
        "simple": "simple",
        "complex": "complex"
    }
)

# 둘 다 END로
branch_workflow.add_edge("simple", END)
branch_workflow.add_edge("complex", END)

# 컴파일
branch_app = branch_workflow.compile()

print("🌿 조건부 분기 워크플로우 완성!")
print("   흐름: START → [길이판단] → simple/complex → END")

In [ ]:
# 조건부 분기 워크플로우 시각화
print("📊 조건부 분기 워크플로우 구조:")
visualize_graph(branch_app)

In [ ]:
# 조건부 분기 동작 테스트
print("🧪 조건부 분기 테스트 - 입력 길이에 따른 경로 선택")
print("📏 기준: 5글자 이하 → simple, 6글자 이상 → complex")
print("=" * 60)

test_cases = [
    "안녕",           # 2글자 → simple
    "안녕하세요!",      # 6글자 → complex
    "LangGraph 최고"  # 12글자 → complex
]

for i, test_input in enumerate(test_cases, 1):
    print(f"\n🧪 테스트 {i}: '{test_input}' ({len(test_input)}글자)")
    
    result = branch_app.invoke({
        "input": test_input,
        "output": "",
        "path": ""
    })
    
    print(f"   🛣️  선택된 경로: {result['path']}")
    print(f"   📝 처리 결과: {result['output']}")

print("\n🎓 조건부 분기로 입력에 따라 다른 처리를 할 수 있어요!")

## 🎓 학습 완료! 기본 개념 정리

### ✅ 성공적으로 배운 것들:

1. **State (상태)** = 워크플로우의 공유 데이터 주머니 📦
   - `TypedDict`로 정의해서 타입 안전성 확보
   - 노드들이 읽고 업데이트하는 공유 저장소

2. **Node (노드)** = State를 처리하는 함수 ⚙️
   - `def my_node(state) -> state:` 형태로 정의
   - 각자 명확한 역할을 가진 처리 단계
   - 독립적으로 테스트 가능

3. **StateGraph** = 노드들을 연결한 워크플로우 🔗
   - `add_node()`로 노드 추가
   - `add_edge()`로 실행 순서 정의
   - `compile()`로 실행 가능한 앱 생성

4. **실행 패턴** = `compile()` → `invoke()` 🚀
   - 그래프를 컴파일해서 실행 가능한 상태로 만들기
   - `invoke(initial_state)`로 워크플로우 실행

5. **조건부 분기** = 입력에 따른 다른 처리 경로 🛣️
   - `set_conditional_entry_point()`로 분기 로직 설정
   - 실무에서 사용자 유형, 문의 종류별 처리에 활용

### 📊 시각화의 힘
- `visualize_graph()`로 워크플로우 구조를 한눈에 파악
- 노드(박스)와 엣지(화살표)로 처리 흐름 직관적 이해

### 🚀 다음 학습 단계
이제 **01-langgraph-fundamentals.ipynb**로 넘어가서:
- 더 상세한 LangGraph 개념과 활용법
- 메모리 시스템으로 대화 지속성 구현  
- ReAct 패턴 이론과 시뮬레이션
- 실무 수준의 워크플로우 구성법

을 배워보세요! 기본기는 완벽해요! 🎉

